In [1]:
# header files
%matplotlib inline
import torch
import numpy as np
import csv
import glob
import math
import tensorboard
from PIL import Image
import skimage
from skimage import io, transform
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
import glob
import csv
import numpy as np
import pickle

plt.rcParams['figure.figsize'] = [15, 15]
print("Header files loaded!")

Header files loaded!


### CPA Training
The CPA Training required the QC_Image.csv file which had the 21 features ['ImageQuality_Correlation_10', 'ImageQuality_Correlation_20', 'ImageQuality_Correlation_5', 'ImageQuality_Correlation_50', 'ImageQuality_FocusScore', 'ImageQuality_LocalFocusScore_10', 'ImageQuality_LocalFocusScore_20', 'ImageQuality_LocalFocusScore_5', 'ImageQuality_LocalFocusScore_50', 'ImageQuality_MADIntensity', 'ImageQuality_MaxIntensity', 'ImageQuality_MeanIntensity', 'ImageQuality_MedianIntensity', 'ImageQuality_MinIntensity', 'ImageQuality_PercentMaximal', 'ImageQuality_PercentMinimal', 'ImageQuality_PowerLogLogSlope', 'ImageQuality_Scaling', 'ImageQuality_StdIntensity', 'ImageQuality_TotalArea' and 'ImageQuality_TotalIntensity'] for the each image used in the training dataset and validation dataset. Three models were tried which will be described in the next section.

In [2]:
# cpa features for the training dataset
files = glob.glob("/home/jupyter-arpit@broadinstitu-ef612/data/broad_annotations/QC_Image.csv")
print(files)

['/home/jupyter-arpit@broadinstitu-ef612/data/broad_annotations/QC_Image.csv']


In [3]:
# get the cpa features for the training and validation dataset
X_train = []
Y_train = []
X_val = []
Y_val = []
train_count_good = 0
train_count_blur = 0
train_count_empty = 0
train_count_debris = 0
val_count_good = 0
val_count_blur = 0
val_count_empty = 0
val_count_debris = 0
for file in files:
    
    flag = -1
    with open(file, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                array = row
                flag = 1
            else:
                array = row
                
                file = array[35]
                if "train" in file:
                    # feature vector
                    train_sample = []
                    for index in range(12, 33):
                        train_sample.append(array[index])
                    X_train.append(train_sample)
                    
                    # label
                    if "good" in file:
                        Y_train.append(0)
                        train_count_good += 1
                    elif "blur" in file:
                        Y_train.append(1)
                        train_count_blur += 1
                    elif "empty" in file:
                        Y_train.append(2)
                        train_count_empty += 1
                    else:
                        Y_train.append(3)
                        train_count_debris += 1
                else:
                    # feature vector
                    val_sample = []
                    for index in range(12, 33):
                        val_sample.append(array[index])
                    X_val.append(val_sample)
                    
                    # label
                    if "good" in file:
                        Y_val.append(0)
                        val_count_good += 1
                    elif "blur" in file:
                        Y_val.append(1)
                        val_count_blur += 1
                    elif "empty" in file:
                        Y_val.append(2)
                        val_count_empty += 1
                    else:
                        Y_val.append(3)
                        val_count_debris += 1
                        

print(len(X_train))
print(len(Y_train))
print(len(X_val))
print(len(Y_val))
print(train_count_good)
print(train_count_blur)
print(train_count_empty)
print(train_count_debris)
print(val_count_good)
print(val_count_blur)
print(val_count_empty)
print(val_count_debris)

2060
2060
400
400
560
600
350
550
100
100
100
100


### CPA Models
The three models used were: Random Forest, AdaBoost and Gradient Boost. The best performing model on the validation dataset was used on the test dataset.<br><br>
The input to the CPA Model -> 21 features of image<br>
The output from the CPA Model -> one of the four classes [good, blurry, empty and debris]

In [4]:
# random-forest classifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, Y_train)


# ada-boost classifier
#rfc = AdaBoostClassifier()
#rfc.fit(X_train, Y_train)


# gradient boosting classifier
#rfc = GradientBoostingClassifier()
#rfc.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [5]:
# prediction on validation dataset
pred = rfc.predict(X_val)
print(pred)
print(Y_val)

[1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 2 3 3 3 3 3 1 1 3 1 1 1 3 1 1 1 1 1 1 3 3 3 0 3 1 0 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 0 3 3 3 3 3 3 3 0 3 3 3 3 3 3 0 3 0 3 3 3 3 3 3 3 3 0 0 3 2 3 3 3
 3 3 3 0 3 2 3 0 3 3 3 2 0 3 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 3 3 2 2 2 2 2
 2 2 2 2 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 3 0 3 0 0 0 0 0 0 3 3 0 3 0 0 0 0 3 0 0 0 0 0 0 0 2 0 0 0 0
 0 3 0 0 0 0 0 3 2 0 0 0 0 0 3 0 0 0 0 0 0 1 3 2 2 3 3 3 1 3]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [6]:
# accuracy and confusion matrix on validation dataset
correct = 0
for index in range(0, 400):
    if pred[index] == Y_val[index]:
        correct += 1
print(correct)
print(confusion_matrix(Y_val, pred))

343
[[79  3  4 14]
 [ 5 86  1  8]
 [ 1  1 95  3]
 [11  3  3 83]]


### Save CPA Model
The best performing model was saved in .sav format. Then it will be loaded and the performance will be analysed on the test dataset.

In [7]:
# save model
filename = '../qc_bestmodel_cpa.sav'
pickle.dump(rfc, open(filename, 'wb'))

### CPA Model on Test Dataset
The next task was to test the performance of the CPA model on the test dataset. The process was three step fold:
1. Load the CPA model
2. Get features for each image in the test dataset
3. Use the CPA model for the prediction

In [8]:
# load saved model
filename = '../qc_bestmodel_cpa.sav'
rfc = pickle.load(open(filename, 'rb'))

In [9]:
# test dataset
files = glob.glob("/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/workspace/qc/2021_03_03_Stain5_CondC_PE_Standard/results/*/*/*_Image.csv")
print(len(files))

17277


In [10]:
test_files = []
X_test = []
for file in files:
    
    flag = -1
    indexes_dna = []
    indexes_rna = []
    indexes_er = []
    indexes_agp = []
    indexes_mito = []
    index_filedna = -1
    index_filerna = -1
    index_fileer = -1
    index_fileagp = -1
    index_filemito = -1
    with open(file, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                array = row
                flag = 1
                for index in range(0, len(array)):
                    if "URL_OrigDNA" in array[index]:
                        index_filedna = index
                    if "URL_OrigRNA" in array[index]:
                        index_filerna = index
                    if "URL_OrigER" in array[index]:
                        index_fileer = index
                    if "URL_OrigMito" in array[index]:
                        index_filemito = index
                    if "URL_OrigAGP" in array[index]:
                        index_fileagp = index
                        
                        
                    # append dna features
                    if "OrigDNA" in array[index] and "ImageQuality" in array[index]:
                        indexes_dna.append(index)
                    
                    # append rna features
                    if "OrigRNA" in array[index] and "ImageQuality" in array[index]:
                        indexes_rna.append(index)
                        
                    # append er features
                    if "OrigER" in array[index] and "ImageQuality" in array[index]:
                        indexes_er.append(index)
                        
                    # append agp features
                    if "OrigAGP" in array[index] and "ImageQuality" in array[index]:
                        indexes_agp.append(index)
                        
                    # append mito features
                    if "OrigMito" in array[index] and "ImageQuality" in array[index]:
                        indexes_mito.append(index)
            else:
                array = row
                
                # dna file
                if index_filedna >= 0:
                    file_dna = "/dgx1nas1/cellpainting-datasets/"
                    sample = array[index_filedna].split("/")
                    for index in range(5, len(sample)):
                        file_dna = file_dna + sample[index]
                        file_dna = file_dna + "/"
                    file_dna = file_dna.replace("2019_07_11_JUMP-CP-pilots", "2019_07_11_JUMP_CP_pilots")
                    test_files.append(file_dna[:len(file_dna)-1])
                    
                    sample_dna = []
                    for index in range(0, len(indexes_dna)):
                        sample_dna.append(array[indexes_dna[index]])
                    X_test.append(sample_dna)
                    
                # rna file
                if index_filerna >= 0:
                    file_rna = "/dgx1nas1/cellpainting-datasets/"
                    sample = array[index_filerna].split("/")
                    for index in range(5, len(sample)):
                        file_rna = file_rna + sample[index]
                        file_rna = file_rna + "/"
                    file_rna = file_rna.replace("2019_07_11_JUMP-CP-pilots", "2019_07_11_JUMP_CP_pilots")
                    test_files.append(file_rna[:len(file_rna)-1])
                    
                    sample_rna = []
                    for index in range(0, len(indexes_rna)):
                        sample_rna.append(array[indexes_rna[index]])
                    X_test.append(sample_rna)
                    
                # er file
                if index_fileer >= 0:
                    file_er = "/dgx1nas1/cellpainting-datasets/"
                    sample = array[index_fileer].split("/")
                    for index in range(5, len(sample)):
                        file_er = file_er + sample[index]
                        file_er = file_er + "/"
                    file_er = file_er.replace("2019_07_11_JUMP-CP-pilots", "2019_07_11_JUMP_CP_pilots")
                    test_files.append(file_er[:len(file_er)-1])
                    
                    sample_er = []
                    for index in range(0, len(indexes_er)):
                        sample_er.append(array[indexes_er[index]])
                    X_test.append(sample_er)
                    
                # agp file
                if index_fileagp >= 0:
                    file_agp = "/dgx1nas1/cellpainting-datasets/"
                    sample = array[index_fileagp].split("/")
                    for index in range(5, len(sample)):
                        file_agp = file_agp + sample[index]
                        file_agp = file_agp + "/"
                    file_agp = file_agp.replace("2019_07_11_JUMP-CP-pilots", "2019_07_11_JUMP_CP_pilots")
                    test_files.append(file_agp[:len(file_agp)-1])
                    
                    sample_agp = []
                    for index in range(0, len(indexes_agp)):
                        sample_agp.append(array[indexes_agp[index]])
                    X_test.append(sample_agp)
                    
                # mito file
                if index_filemito >= 0:
                    file_mito = "/dgx1nas1/cellpainting-datasets/"
                    sample = array[index_filemito].split("/")
                    for index in range(5, len(sample)):
                        file_mito = file_mito + sample[index]
                        file_mito = file_mito + "/"
                    file_mito = file_mito.replace("2019_07_11_JUMP-CP-pilots", "2019_07_11_JUMP_CP_pilots")
                    test_files.append(file_mito[:len(file_mito)-1])
                    
                    sample_mito = []
                    for index in range(0, len(indexes_mito)):
                        sample_mito.append(array[indexes_mito[index]])
                    X_test.append(sample_mito)
print(len(X_test))
print(len(test_files))

86385
86385


In [11]:
pred = rfc.predict(X_test)
pred_prob = rfc.predict_proba(X_test)
print(pred)

[0 0 0 ... 0 0 0]


### Save Predictions
After running the CPA model through the test dataset, the next task was to use the predictions on each image of the test dataset and check the count for each class and save it in a csv file.

In [12]:
count_good = 0
count_blur = 0
count_empty = 0
count_debris = 0
for index in range(0, len(pred)):
    if pred[index] == 0:
        count_good += 1
    elif pred[index] == 1:
        count_blur += 1
    elif pred[index] == 2:
        count_empty += 1
    elif pred[index] == 3:
        count_debris += 1

print(count_good)
print(count_blur)
print(count_empty)
print(count_debris)

61509
0
54
24822


In [13]:
# write results
count = 0
with open("test_baseline_cpa.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(["File Path", "Good Probability", "Blurry Probability", "Empty Probability", "Debris Probability", "Label"])
    for index in range(0, len(pred)):
        if pred[index] == 1 or pred[index] == 2 or pred[index] == 3 or pred[index] == 0:
            count += 1
            spamwriter.writerow([test_files[index], pred_prob[index][0], pred_prob[index][1], pred_prob[index][2], pred_prob[index][3], pred[index]])
print(count)

86385
